In [1]:
import numpy as np
import pandas as pd
import json
import os

In [3]:
folder_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\DE Optimization\popsizeFiles"
potential = 'DW'
cutoff = 16

dataframes = {}

folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

converged_runs = []
mean_iters = []
min_e = []
delta_min_e = []
median_e = []
delta_median_e = []
exact_e = []
time_taken = []

num_vqe = []
shots = []
tolerance = []
abs_tolerance = []
strategy = []
popsize = []
maxiter = []
init = []


for f in folders:

    data_path = folder_path + "\\{}\\{}_{}.json".format(f,potential, cutoff)
    
    with open(data_path, 'r') as file:
        data = json.load(file)

    num_vqe.append(data['num_VQE'])
    shots.append(data['shots'])

    optimzer = data['Optimizer']
    maxiter.append(optimzer['maxiter'])
    #tolerance.append(optimzer['tolerance'])
    #abs_tolerance.append(optimzer['abs_tolerance'])
    #strategy.append(optimzer['strategy'])
    popsize.append(optimzer['popsize'])
    init.append(optimzer['init'])

    #success = sum(data['success'])
    
    exact_min_e = np.min(data['exact_eigenvalues'])
    exact_e.append(exact_min_e)

    converged_runs.append(sum(data['success']))
    mean_iters.append(int(np.round(np.mean(data['num_iters']))))
    min_e.append(np.min(data['results']))
    delta_min_e.append(abs(exact_min_e - np.min(data['results'])))
    median_e.append(np.median(data['results']))
    delta_median_e.append(abs(exact_min_e - np.median(data['results'])))
    
    time_taken.append(data['total_VQE_time'])

    

In [7]:
df = pd.DataFrame({
    'popsize': folders,
    'num_vqe': num_vqe,
    'shots': shots,
    #'tolerance': tolerance,
    #'abs_tolerance': abs_tolerance,
    #'strategy': strategy,
    #'popsize': popsize,
    'init': init,
    'Converged Runs': [f"{c}/40" for c in converged_runs],
    'maxiter': maxiter,
    'Mean Iter': mean_iters,
    #'VQE min E': [f"{num:.5e}" for num in min_e],
    'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    #'VQE median E': [f"{num:.5e}" for num in median_e],
    'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    'Exact': [f"{num:.5e}" for num in exact_e],
    'Time': time_taken
})

In [8]:
df.sort_values(['Delta min E', 'Delta median E'], ascending=[False,False])

,popsize,num_vqe,shots,init,Converged Runs,maxiter,Mean Iter,Delta min E,Delta median E,Exact,Time
2,20,40,1024,custom,40/40,10000,143,7.31637e-14,4.15225e-05,8.91599e-01,0:10:10.726670
3,25,40,1024,custom,40/40,10000,183,7.31637e-14,4.15022e-05,8.91599e-01,0:14:41.362001
4,30,40,1024,custom,40/40,10000,141,7.31637e-14,4.15022e-05,8.91599e-01,0:14:25.687482
0,10,40,1024,custom,38/40,10000,933,7.31637e-14,2.43449e-02,8.91599e-01,0:24:01.104046
1,15,40,1024,custom,40/40,10000,316,7.31637e-14,1.82009e-03,8.91599e-01,0:13:37.611176
5,5,40,1024,custom,32/40,10000,2652,2.03387e-01,2.04944e+00,8.91599e-01,0:34:13.146936
